In [1]:
import xtrack as xt
import numpy as np
import scipy.constants as sc
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import mplcursors
%matplotlib tk

# # The following code is to allow multiple figures into one as tabs
# import matplotlib
# # prevent NoneType error for versions of matplotlib 3.1.0rc1+ by calling matplotlib.use()
# # For more on why it's nececessary, see
# # https://stackoverflow.com/questions/59656632/using-qt5agg-backend-with-matplotlib-3-1-2-get-backend-changes-behavior
# matplotlib.use('qt5agg')

# from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
# from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
# from PyQt5.QtWidgets import QMainWindow, QApplication, QWidget, QTabWidget, QVBoxLayout
# import matplotlib.pyplot as plt
# import sys

# class plotWindow():
#     def __init__(self, parent=None):
#         self.app = QApplication(sys.argv)
#         self.MainWindow = QMainWindow()
#         self.MainWindow.__init__()
#         self.MainWindow.setWindowTitle("plot window")
#         self.canvases = []
#         self.figure_handles = []
#         self.toolbar_handles = []
#         self.tab_handles = []
#         self.current_window = -1
#         self.tabs = QTabWidget()
#         self.MainWindow.setCentralWidget(self.tabs)
#         self.MainWindow.resize(1280, 900)
#         self.MainWindow.show()

#     def addPlot(self, title, figure):
#         new_tab = QWidget()
#         layout = QVBoxLayout()
#         new_tab.setLayout(layout)

#         figure.subplots_adjust(left=0.05, right=0.99, bottom=0.05, top=0.91, wspace=0.2, hspace=0.2)
#         new_canvas = FigureCanvas(figure)
#         new_toolbar = NavigationToolbar(new_canvas, new_tab)

#         layout.addWidget(new_canvas)
#         layout.addWidget(new_toolbar)
#         self.tabs.addTab(new_tab, title)

#         self.toolbar_handles.append(new_toolbar)
#         self.canvases.append(new_canvas)
#         self.figure_handles.append(figure)
#         self.tab_handles.append(new_tab)

#     def show(self):
#         self.app.exec_()

In [2]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta4.json')
line['qph_setvalue'] = 0.5
line['qpv_setvalue'] = 0.5
line_no_aper = xt.Line(elements=[], element_names=[])

for ee, nn in zip(line.elements, line.element_names):
    if xt.line._is_aperture(ee, line):
        line_no_aper.append_element(xt.Marker(), nn)
    else:
        line_no_aper.append_element(ee, nn)
        # We are missing the parents of the slices
        if hasattr(ee, 'parent_name'):
            # If the parent is not in the line, we add it
            if ee.parent_name not in line_no_aper.element_dict:
                line_no_aper.env.element_dict[ee.parent_name] = line.get(ee.parent_name)
line_no_aper.particle_ref = line.particle_ref


Loading line from dict:   0%|          | 0/36395 [00:00<?, ?it/s]

Done loading line from dict.           


In [3]:
# Calculate bucket height
tw_ref = line.twiss()
beta0 = line.particle_ref.beta0[0]
E = line.particle_ref.energy0[0]
q = line.particle_ref.q0
f_RF = 200e6
h = f_RF * line.get_length() / beta0 / sc.c
eta = tw_ref.slip_factor

V_RF = 3e6
bucket_height_3MV = np.sqrt(2*q*V_RF / (np.pi*h*abs(eta)*E*beta0**2))
print(f"{bucket_height_3MV=}")

V_RF = 4.5e6
bucket_height_4_5MV = np.sqrt(2*q*V_RF / (np.pi*h*abs(eta)*E*beta0**2))
print(f"{bucket_height_4_5MV=}")

bucket_height_3MV=0.0029838516102516405
bucket_height_4_5MV=0.0036544569566492316


In [4]:
# Perform a bunch of twisses for different delta0
tw = {}
beam_sizes = {}
nemitt = 2e-6
for delta in np.linspace(-0.0076, 0.0076, 101):
    this_delta = round(delta,6)
    tw[this_delta] = line_no_aper.twiss(delta0=delta)
    beam_sizes[this_delta] = tw[this_delta].get_beam_covariance(nemitt_x=nemitt, nemitt_y=nemitt)


In [5]:
plt.plot(*np.array([[dd, tww.rows['qd.12510'].x[0]] for dd, tww in tw.items()]).T)
plt.plot(*np.array([[-dd, -tww.rows['qd.12510'].x[0]] for dd, tww in tw.items()]).T)

In [5]:
def get_aper(ee):
    if isinstance(ee, xt.LimitPolygon):
        raise NotImplementedError
    max_x = 100
    min_x = -100
    max_y = 100
    min_y = -100
    if hasattr(ee, 'max_x'):
        max_x = ee.max_x
    if hasattr(ee, 'min_x'):
        min_x = ee.min_x
    if hasattr(ee, 'a_squ'):
        max_x = min(np.sqrt(ee.a_squ), max_x)
        min_x = max(-np.sqrt(ee.a_squ), min_x)
    if hasattr(ee, 'max_y'):
        max_y = ee.max_y
    if hasattr(ee, 'min_y'):
        min_y = ee.min_y
    if hasattr(ee, 'b_squ'):
        max_y = min(np.sqrt(ee.b_squ), max_y)
        min_y = max(-np.sqrt(ee.b_squ), min_y)
    if hasattr(ee, '_sin_rot_s') and -2 < ee._sin_rot_s < 2:
        old_max_x = max_x
        max_x = ee._cos_rot_s * max_x + ee._sin_rot_s * max_y
        max_y = -ee._sin_rot_s * old_max_x + ee._cos_rot_s * max_y
    if hasattr(ee, '_shift_x'):
        max_x += ee._shift_x
        min_x += ee._shift_x
    if hasattr(ee, '_shift_y'):
        max_y += ee._shift_y
        min_y += ee._shift_y
    return max_x, min_x, max_y, min_y

tt = line.get_table()
tt_aper = tt.rows[[ttt.startswith('Limit') for ttt in tt.element_type]].cols['name s']
tt_aper.aper_max_x, tt_aper.aper_min_x, tt_aper.aper_max_y, tt_aper.aper_min_y = \
    np.array([list(get_aper(line[nn]))  for nn in tt_aper.name]).T

In [7]:
n_limit = 20
delta = np.array(list(tw.keys()))
aper_L = np.array([(tt_aper.aper_max_x - tw[dd].rows[tt_aper.name].x) / beam_sizes[dd].rows[tt_aper.name].sigma_x for dd in delta]).T
aper_R = np.array([(tt_aper.aper_min_x - tw[dd].rows[tt_aper.name].x) / beam_sizes[dd].rows[tt_aper.name].sigma_x for dd in delta]).T
aper_L = aper_L[[np.any((aa < n_limit) & (aa > -n_limit)) for aa in aper_L]]
aper_R = aper_R[[np.any((aa < n_limit) & (aa > -n_limit)) for aa in aper_R]]

fig, ax = plt.subplots(figsize=(8, 5))

figs = []

# 1) Make a custom cyclic colormap: green → red → green
cmap = mcolors.LinearSegmentedColormap.from_list(
    "green_red_cycle",
    ["tab:green", "tab:red", "tab:green"]
)
# 2) Normalize s so 0→7000 maps to 0→1
norm = mcolors.Normalize(vmin=0, vmax=line.get_length())

# plot left apertures
for idx, data in enumerate(aper_L):
    # grab the Line2D object
    fig, = ax.plot(1000*delta, data, color=cmap(norm(tt_aper.s[idx])), picker=5)
    # stash the name on the fig’s "label" field
    fig.set_label(f"{tt_aper.name[idx]}  ({tt_aper.s[idx]}m)")
    figs.append(fig)

# plot right apertures
for idx, data in enumerate(aper_R):
    fig, = ax.plot(1000*delta, data, color=cmap(norm(tt_aper.s[idx])), picker=5)
    fig.set_label(f"{tt_aper.name[idx]}  ({tt_aper.s[idx]}m)")
    figs.append(fig)

ax.set_xlabel(r"$\delta\; [10^{-3}] $")
ax.set_ylabel(r"$x\; [\sigma_{2\mu\text{m}}] $")
ax.set_ylim((-n_limit,n_limit))
ax.vlines(-1000*bucket_height_3MV, -n_limit, n_limit, color='gray', linestyle='--')
ax.vlines(1000*bucket_height_3MV, -n_limit, n_limit, color='gray', linestyle='--')
ax.vlines(-2*1000*bucket_height_3MV, -n_limit, n_limit, color='lightgray', linestyle='--')
ax.vlines(2*1000*bucket_height_3MV, -n_limit, n_limit, color='lightgray', linestyle='--')

# # now attach the hover‐tootip
# cursor = mplcursors.cursor(figs)
# @cursor.connect("add")
# def _(sel):
#     sel.annotation.set_text(sel.artist.get_label())

plt.savefig('sps_stacked_aper.png', dpi=300)
plt.show()

In [ ]:
# for i in range(6):
#     fig, ax = plt.subplots(figsize=(20, 5))
#     delta = list(tw.keys())
#     for dd in delta:
#         if abs(dd) > 0.00375:
#             ax.plot(tw[dd].s, tw[dd].x + np.sign(dd)*2*i*beam_sizes[dd].sigma_x, 'r-')
#         else:
#             ax.plot(tw[dd].s, tw[dd].x + np.sign(dd)*2*i*beam_sizes[dd].sigma_x, 'g-')
#     line1, = ax.plot(tt_aper.s, tt_aper.aper_max_x, 'k-')
#     line2, = ax.plot(tt_aper.s, tt_aper.aper_min_x, 'k-')
#     ax.set_xlabel('s [m]')
#     ax.set_ylabel('Aperture $x$ [m]')
#     ax.set_title(f'Betatron {2*i}' + r'$\sigma_x$   $\delta$ within 1 bucket (green) and 2 buckets (red)')
#     cursor = mplcursors.cursor([line1, line2])
#     @cursor.connect("add")
#     def on_add(sel):
#         if hasattr(sel, 'index'):
#             raw_idx = sel.index
#         else:
#             raw_idx = sel.target.index
#         idx = int(raw_idx)
#         sel.annotation.set_text(tt_aper.name[idx])
#     plt.show()

In [ ]:
pw = plotWindow()
for i in range(6):
    fig, ax = plt.subplots(figsize=(20, 4))
    delta = list(tw.keys())
    for dd in delta:
        # if abs(dd) < 0.00375:
        ax.plot(tw[dd].s, tw[dd].x + np.sign(dd)*2*i*beam_sizes[dd].sigma_x, 'r-')
        # else:
        #     ax.plot(tw[dd].s, tw[dd].x + np.sign(dd)*2*i*beam_sizes[dd].sigma_x, 'g-')
    line1, = ax.plot(tt_aper.s, tt_aper.aper_max_x, 'k-')
    line2, = ax.plot(tt_aper.s, tt_aper.aper_min_x, 'k-')
    ax.set_xlabel('s [m]')
    ax.set_ylabel('Aperture $x$ [m]')
    ax.set_title(f'Betatron {2*i}' + r'$\sigma_x$   $\delta$ within 1 bucket (green) and 2 buckets (red)')
    cursor = mplcursors.cursor([line1, line2])
    @cursor.connect("add")
    def on_add(sel):
        if hasattr(sel, 'index'):
            raw_idx = sel.index
        else:
            raw_idx = sel.target.index
        idx = int(raw_idx)
        sel.annotation.set_text(tt_aper.name[idx])
    pw.addPlot(f"{2*i}" + r'$\sigma_x$', fig)
pw.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 4))
# fig, ax = plt.subplots(figsize=(13.33, 6))
delta = list(tw.keys())
for dd in delta:
    if abs(dd) < 0.00375:
        ax.plot(tw[dd].s, tw[dd].x + np.sign(dd)*5*beam_sizes[dd].sigma_x, 'g-')
    else:
        ax.plot(tw[dd].s, tw[dd].x + np.sign(dd)*5*beam_sizes[dd].sigma_x, 'r-')
line1, = ax.plot(tt_aper.s, tt_aper.aper_max_x, 'k-')
line2, = ax.plot(tt_aper.s, tt_aper.aper_min_x, 'k-')
ax.set_xlabel('s [m]')
ax.set_ylabel('Aperture $x$ [m]')
ax.set_title(f'Betatron {5}' + r'$\sigma_x$   $\delta$ within 1 bucket (green) and 2 buckets (red)')
cursor = mplcursors.cursor([line1, line2])
@cursor.connect("add")
def on_add(sel):
    if hasattr(sel, 'index'):
        raw_idx = sel.index
    else:
        raw_idx = sel.target.index
    idx = int(raw_idx)
    sel.annotation.set_text(tt_aper.name[idx])
# ax.set_xlim(224,480)
# ax.set_ylim(-0.1,0.1)
# plt.savefig('sps_aper_around_qd.11110.png', dpi=300)
plt.show()

: 

In [6]:
tt.rows['q.*'].show()

name                              s element_type             isthick isreplica parent_name ...
qf.10010                          0 Quadrupole                  True     False None       
qd.10110                    31.9977 Quadrupole                  True     False None       
qf.10210                    63.9954 Quadrupole                  True     False None       
qd.10310                    95.9931 Quadrupole                  True     False None       
qf.10410                    127.991 Quadrupole                  True     False None       
qd.10510                    159.988 Quadrupole                  True     False None       
qf.10610                    191.986 Quadrupole                  True     False None       
qd.10710                    223.984 Quadrupole                  True     False None       
qf.10810                    255.982 Quadrupole                  True     False None       
qd.10910                    287.979 Quadrupole                  True     False None   

In [ ]:
line.element_names[-40:]

('vttb.63602.b_aper',
 'drift_1901..3',
 'loe.63602_entry',
 'loe.63602..entry_map',
 'loe.63602..0',
 'loe.63602.a_aper',
 'loe.63602..1',
 'loe.63602.b_aper',
 'loe.63602..2',
 'loe.63602..exit_map',
 'loe.63602_exit',
 'drift_1902',
 'lsf.63605_entry',
 'lsf.63605..entry_map',
 'lsf.63605..0',
 'lsf.63605.a_aper',
 'lsf.63605..1',
 'lsf.63605.b_aper',
 'lsf.63605..2',
 'lsf.63605..exit_map',
 'lsf.63605_exit',
 'drift_1903..0',
 'vttb.63602.c_aper',
 'drift_1903..1',
 'drift_mdh.63607..1..0',
 'mdh.63607.a_aper',
 'drift_mdh.63607..1..1',
 'mdh.63607',
 'drift_mdh.63607..2..0',
 'mdh.63607.b_aper',
 'drift_mdh.63607..2..1',
 'drift_1904',
 'bph.63608..0',
 'bph.63608.a_aper',
 'bph.63608..1',
 'bph.63608.b_aper',
 'bph.63608..2',
 'drift_1905',
 'end.10010',
 'sps$end')

In [ ]:
pos = 'qf.11010'
tt.rows[f'{pos}<<40':f'{pos}'].show()

name                              s element_type       isthick isreplica parent_name        ...
mba.10990..4                313.868 ThickSliceRBend       True     False mba.10990         
mba.10990..5_aper2          314.494 LimitRect            False     False None              
mba.10990..5                314.494 ThickSliceRBend       True     False mba.10990         
mba.10990..6_aper2           315.12 LimitRect            False     False None              
mba.10990..6                 315.12 ThickSliceRBend       True     False mba.10990         
mba.10990..7_aper2          315.746 LimitRect            False     False None              
mba.10990..7                315.746 ThickSliceRBend       True     False mba.10990         
mba.10990..8_aper2          316.372 LimitRect            False     False None              
mba.10990..8                316.372 ThickSliceRBend       True     False mba.10990         
mba.10990..9_aper2          316.998 LimitRect            False     False Non

: 

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
lines = []
this_tw = tw[-0.0038].rows['qf.*']
line, = ax.plot(this_tw.s, this_tw.x, 'b.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'b.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'b.'); lines.append(line)
this_tw = tw[-0.0076].rows['qf.*']
line, = ax.plot(this_tw.s, this_tw.x, 'b.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'b.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'b.'); lines.append(line)
this_tw = tw[-0.0038].rows['qd.*']
line, = ax.plot(this_tw.s, this_tw.x, 'm.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'm.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'm.'); lines.append(line)
this_tw = tw[-0.0076].rows['qd.*']
line, = ax.plot(this_tw.s, this_tw.x, 'm.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'm.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'm.'); lines.append(line)
this_tw = tw[-0.0038].rows['mba.*']
line, = ax.plot(this_tw.s, this_tw.x, 'g.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'g.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'g.'); lines.append(line)
this_tw = tw[-0.0076].rows['mba.*']
line, = ax.plot(this_tw.s, this_tw.x, 'g.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'g.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'g.'); lines.append(line)
this_tw = tw[-0.0038].rows['mbb.*']
line, = ax.plot(this_tw.s, this_tw.x, 'r.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'r.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'r.'); lines.append(line)
this_tw = tw[-0.0076].rows['mbb.*']
line, = ax.plot(this_tw.s, this_tw.x, 'r.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 5*np.sqrt(this_tw.betx *2e-6/28), 'r.'); lines.append(line)
line, = ax.plot(this_tw.s, this_tw.x - 10*np.sqrt(this_tw.betx *2e-6/28), 'r.'); lines.append(line)
ax.hlines(-7.6e-2, 0, 7000, color='b', linestyle='--') # QF
ax.hlines(-4e-2, 0, 7000, color='m', linestyle='--') # QD
ax.hlines(-7.6e-2-2.2e-3, 0, 7000, color='g', linestyle='--') # MBA
ax.hlines(-7.6e-2+4.4e-3, 0, 7000, color='g', linestyle='--') # MBA
ax.hlines(-6.45e-2-2.2e-3, 0, 7000, color='r', linestyle='--') # MBB
ax.hlines(-6.45e-2+4.4e-3, 0, 7000, color='r', linestyle='--') # MBB

cursor = mplcursors.cursor(lines)
@cursor.connect("add")
def on_add(sel):
    if hasattr(sel, 'index'):
        raw_idx = sel.index
    else:
        raw_idx = sel.target.index
    idx = int(raw_idx)
    sel.annotation.set_text(tw[0].rows['q[df].*'].name[idx])
plt.show()

In [ ]:
tw.keys()

dict_keys([-0.0076, -0.007448, -0.007296, -0.007144, -0.006992, -0.00684, -0.006688, -0.006536, -0.006384, -0.006232, -0.00608, -0.005928, -0.005776, -0.005624, -0.005472, -0.00532, -0.005168, -0.005016, -0.004864, -0.004712, -0.00456, -0.004408, -0.004256, -0.004104, -0.003952, -0.0038, -0.003648, -0.003496, -0.003344, -0.003192, -0.00304, -0.002888, -0.002736, -0.002584, -0.002432, -0.00228, -0.002128, -0.001976, -0.001824, -0.001672, -0.00152, -0.001368, -0.001216, -0.001064, -0.000912, -0.00076, -0.000608, -0.000456, -0.000304, -0.000152, 0.0, 0.000152, 0.000304, 0.000456, 0.000608, 0.00076, 0.000912, 0.001064, 0.001216, 0.001368, 0.00152, 0.001672, 0.001824, 0.001976, 0.002128, 0.00228, 0.002432, 0.002584, 0.002736, 0.002888, 0.00304, 0.003192, 0.003344, 0.003496, 0.003648, 0.0038, 0.003952, 0.004104, 0.004256, 0.004408, 0.00456, 0.004712, 0.004864, 0.005016, 0.005168, 0.00532, 0.005472, 0.005624, 0.005776, 0.005928, 0.00608, 0.006232, 0.006384, 0.006536, 0.006688, 0.00684, 0.0069